In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [2]:
# 데이터로드 (ch4-2(붓꽃데이터).csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('datasets/국민건강정보/NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [3]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [4]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [5]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [6]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx

[2,
 8,
 30,
 67,
 75,
 77,
 80,
 115,
 116,
 148,
 167,
 190,
 192,
 197,
 211,
 252,
 266,
 292,
 308,
 346,
 358,
 368,
 377,
 379,
 385,
 394,
 399,
 413,
 415,
 424,
 458,
 466,
 467,
 471,
 508,
 514,
 536,
 571,
 587,
 592,
 597,
 648,
 649,
 656,
 672,
 676,
 684,
 714,
 752,
 753,
 797,
 824,
 843,
 844,
 849,
 857,
 862,
 879,
 927,
 930,
 955,
 967,
 975,
 1009,
 1048,
 1100,
 1118,
 1120,
 1150,
 1160,
 1189,
 1255,
 1264,
 1269,
 1290,
 1295,
 1324,
 1338,
 1339,
 1343,
 1348,
 1351,
 1363,
 1391,
 1400,
 1413,
 1423,
 1440,
 1529,
 1534,
 1558,
 1559,
 1567,
 1608,
 1646,
 1655,
 1716,
 1718,
 1719,
 1721,
 1730,
 1733,
 1740,
 1747,
 1769,
 1798,
 1835,
 1854,
 1868,
 1871,
 1880,
 1886,
 1898,
 1906,
 1912,
 1960,
 1962,
 1973,
 1987,
 2034,
 2095,
 2132,
 2156,
 2213,
 2262,
 2270,
 2287,
 2312,
 2381,
 2397,
 2410,
 2476,
 2521,
 2589,
 2645,
 2654,
 2669,
 2680,
 2681,
 2704,
 2712,
 2743,
 2770,
 2776,
 2781,
 2815,
 2816,
 2826,
 2832,
 2833,
 2913,
 2929,
 2933,
 

In [7]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,1,8,175.0,75.0,94.0,1.0,0.8,1.0,1.0,130.0,...,271.0,64.0,92.0,1.0,1.1,45.0,62.0,136.0,1.0,0.0
1,1,8,170.0,80.0,93.0,0.7,0.9,1.0,1.0,115.0,...,185.0,32.0,89.0,1.0,1.0,31.0,29.0,51.0,2.0,0.0
2,1,10,160.0,60.0,84.0,1.0,0.9,1.0,1.0,128.0,...,145.0,44.0,152.0,1.0,1.2,29.0,35.0,24.0,2.0,0.0
3,1,7,175.0,75.0,78.0,0.9,0.9,1.0,1.0,108.0,...,114.0,53.0,106.0,1.0,1.2,24.0,27.0,35.0,1.0,1.0
4,1,8,165.0,65.0,84.0,1.2,1.2,1.0,1.0,126.0,...,40.0,72.0,79.0,1.0,1.1,22.0,16.0,12.0,1.0,0.0
5,1,8,180.0,80.0,83.5,1.5,1.2,1.0,1.0,129.0,...,56.0,70.0,111.0,1.0,1.1,19.0,24.0,21.0,1.0,1.0
6,1,11,165.0,55.0,74.0,0.9,0.8,1.0,1.0,100.0,...,47.0,45.0,82.0,1.0,0.8,23.0,16.0,23.0,3.0,0.0
7,1,8,170.0,70.0,83.0,1.5,0.7,1.0,1.0,106.0,...,478.0,37.0,132.0,1.0,0.9,35.0,47.0,176.0,1.0,1.0
8,1,9,175.0,85.0,93.0,1.5,1.5,1.0,1.0,128.0,...,268.0,54.0,141.0,1.0,1.2,24.0,28.0,28.0,2.0,1.0
9,1,6,175.0,85.0,90.0,1.5,1.5,1.0,1.0,117.0,...,42.0,57.0,90.0,1.0,0.8,19.0,22.0,15.0,1.0,1.0


In [8]:
rawData_sample.shape

(50000, 22)

In [9]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [10]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
from sklearn.ensemble import RandomForestClassifier  # 랜덤포리스트 기법에 관련된 모듈

# DecisionTreeClassifier() : 의사결정트리를 생성하는 함수
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
random_forest

RandomForestClassifier(n_estimators=20, random_state=0)

In [12]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
random_forest.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = random_forest.predict(X_test)

In [13]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.72


In [14]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.72


In [15]:
# RFE(Recursive Feature Elimination)
from sklearn.feature_selection import RFE

select = RFE(RandomForestClassifier(n_estimators=20, random_state=0), n_features_to_select = 10)
select.fit(X_train, y_train)
select

RFE(estimator=RandomForestClassifier(n_estimators=20, random_state=0),
    n_features_to_select=10)

In [16]:
X_train_RFE = select.transform(X_train)
X_test_RFE = select.transform(X_test)
random_forest.fit(X_train_RFE, y_train)

RandomForestClassifier(n_estimators=20, random_state=0)

In [17]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : select.ranking_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=True)

,중요도
흡연상태,1
연령대코드(5세단위),1
신장(5Cm단위),1
허리둘레,1
LDL콜레스테롤,1
HDL콜레스테롤,1
트리글리세라이드,1
수축기혈압,1
감마지티피,1
식전혈당(공복혈당),1


In [18]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train_RFE, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test_RFE, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.71


In [19]:
# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
y_pred_tr = random_forest.predict(X_test_RFE)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.71
